# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure G Maps

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the dataframe into a variable for manipulation
df = pd.read_csv("../output_data/cities.csv").drop("Unnamed: 0",axis = 1)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#Set Coordinates 
coordinates = df[["Lat", "Lng"]]

#configure the figure layout
figure_layout = {
    'width': '1100px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Create the figure variable
fig = gmaps.figure(layout=figure_layout)

# Add Heat Layer
heat_layer = gmaps.heatmap_layer(coordinates, weights= df["Humidity"].astype(float),max_intensity = 100, dissipating=False, point_radius=2)
fig.add_layer(heat_layer)

# Show Fig
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create new dataframe based off the new conditions
new_frame = df.loc[(df["Max Temp"]> 70) & (df["Max Temp"] < 80)].loc[df["Wind Speed"] < 10].loc[df["Cloudiness"] == 0].dropna()

# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

new_frame

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
126,Bubaque,11.2833,-15.8333,79.99,56,0,8.12,GW,1612747890
152,Harper,4.3750,-7.7169,79.20,89,0,6.55,LR,1612747378
182,Port Pirie,-33.1833,138.0167,71.60,53,0,8.05,AU,1612747672
241,Tsabong,-26.0217,22.4010,72.36,75,0,8.37,BW,1612747901
256,Paso de los Toros,-32.8167,-56.5167,72.75,51,0,3.06,UY,1612747902
282,Hambantota,6.1241,81.1185,73.40,100,0,2.30,LK,1612747375
387,Barra de Tecoanapa,16.5167,-98.7500,77.29,78,0,7.29,MX,1612747915
464,Bull Savanna,17.8868,-77.5902,71.44,81,0,8.95,JM,1612747924
556,Kon Tum,14.3500,108.0000,76.68,59,0,3.96,VN,1612747933
582,Cidreira,-30.1811,-50.2056,70.66,72,0,7.40,BR,1612747361


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = new_frame
hotel = []
hotel_df

for index, row in hotel_df.iterrows():
    location = str(row["Lat"]) + "," + str(row["Lng"])
    params = {
    "location": location,
    "keyword": "hotel",
    "radius": "5000",
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Start try except incase a hotel cannot be located nearby
    try:
        response = requests.get(base_url, params=params).json()
        hotel.append(response["results"][0]["name"])
    
    except:
        hotel.append("None Nearby")

#add "Hotel Name" column to hotel
hotel_df["Hotel Name"] = hotel

In [6]:
# add hotel column to df
hotel_df["Hotel Name"] = hotel
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
126,Bubaque,11.2833,-15.8333,79.99,56,0,8.12,GW,1612747890,Le Mango Lodge
152,Harper,4.3750,-7.7169,79.20,89,0,6.55,LR,1612747378,Grands' Guesthouse
182,Port Pirie,-33.1833,138.0167,71.60,53,0,8.05,AU,1612747672,Ellen Hotel
241,Tsabong,-26.0217,22.4010,72.36,75,0,8.37,BW,1612747901,Dikukama Hotel
256,Paso de los Toros,-32.8167,-56.5167,72.75,51,0,3.06,UY,1612747902,Hotel FK Paso de los Toros
282,Hambantota,6.1241,81.1185,73.40,100,0,2.30,LK,1612747375,The Peacock Beach Hotel Hambantota
387,Barra de Tecoanapa,16.5167,-98.7500,77.29,78,0,7.29,MX,1612747915,Hotel Garapacho
464,Bull Savanna,17.8868,-77.5902,71.44,81,0,8.95,JM,1612747924,Eves rock away villa
556,Kon Tum,14.3500,108.0000,76.68,59,0,3.96,VN,1612747933,Hotels Kontum
582,Cidreira,-30.1811,-50.2056,70.66,72,0,7.40,BR,1612747361,Hotel Castelo


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Create gmaps figure variable
fig2 = gmaps.figure(layout=figure_layout)

# Add heatlayer
heat_layer = gmaps.heatmap_layer(coordinates, weights= df["Humidity"].astype(float),max_intensity = 100, dissipating=False, point_radius=2)
fig2.add_layer(heat_layer)

# Add markers and info box
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2.add_layer(markers)

#print the figure
fig2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…